# DFT calculations in DFTK

In [ ]:
# Activate source directory of DFTK
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using DFTK
using Plots

### Parameters

In [ ]:
kgrid = [3, 3, 3]    # k-Point grid
Ecut = 15            # Kinetic energy cutoff in Hartree
temperature = 0.01   # Smearing temperature in Hartree (for metals)
T = Float64;         # Floating point type for computations

### Setting up lattices and structures

In [ ]:
# Build Silicon (an Insulator)

a = 10.263141334305942  # Silicon lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]
n_bands = 6

model = model_LDA(Array{T}(lattice), atoms);

or to have full control over the employed terms:

In [ ]:
terms = [
    Kinetic(),
    AtomicLocal(),
    AtomicNonlocal(),
    Ewald(),
    PspCorrection(),
    Hartree(),
    Xc(:lda_x, :lda_c_vwn)
]
model = Model(Array{T}(lattice), atoms=atoms, terms=terms);

or

In [ ]:
# Build graphite (a Metal)
# Note: This is not exactly the minimum-energy structure

ÅtoBohr = 1.8897261246257702  # Convert Ångström to bohr
a = 1.228ÅtoBohr   
b = 2.12695839ÅtoBohr
c = 7ÅtoBohr
lattice = [[a a 0]; [-b b 0]; [0 0 c]]
C = ElementPsp(:C, psp=load_psp("hgh/lda/C-q4"))
atoms = [C => [[0, 0, 1/4], [0, 0, 3/4],
               [1/3, 2/3, 1/4], [2/3, 1/3, 3/4]], ]
n_bands = 15

model = model_DFT(Array{T}(lattice), atoms, :lda_xc_teter93;
                  temperature=temperature, smearing=Smearing.MethfesselPaxton2());

### Running an SCF computation

In [ ]:
# Discretize the model
basis = PlaneWaveBasis(model, Ecut, kgrid=kgrid);

In [ ]:
# Run self-consistent field calculation
scfres = self_consistent_field(basis, tol=1e-10)

display(scfres.energies)
println("\nkpt   occupation")
for ik in 1:length(basis.kpoints)
    println("   $ik  $(scfres.occupation[ik])")
end

### Post-process SCF results

In [ ]:
# Compute bandstructure
plot_bandstructure(scfres, n_bands)

In [ ]:
# Plot of the density versus the norm of the reciprocal lattice points

Gabs = vec([sum(abs, G) for G in G_vectors(basis)])
scatter(Gabs, vec(abs.(scfres.ρ.fourier)), yaxis=:log)

In [ ]:
# Plot the density of state
εs = collect(-0.1:0.005:0.5)
doses = DOS.(εs, Ref(basis), Ref(scfres.eigenvalues), T=temperature*4,
             smearing=DFTK.Smearing.MethfesselPaxton1())
q = plot(εs, doses, label="DOS")
vline!(q, [scfres.εF], label="Fermi level")